In [ ]:
from google.colab import drive#
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install transformers
from transformers import pipeline

In [ ]:
from tqdm import tqdm

In [ ]:
df = pd.read_csv('Your_data.csv')
classifier = pipeline("zero-shot-classification",device=0) #download the pretrained model - sr device = 0 to use GPU
candidate_labels = ['Media coverage','Activism','Police brutality events','No response from public figures','Questions about moving forward/past','Lekki Massacre','Irrelevant']

In [ ]:
predictedCategories = []
#categories_scores = []
tweet_scores = []
for i in tqdm(range(len(df))):
    text = df.iloc[i,]['cleaned_tweets']
    res = classifier(text, candidate_labels, multi_class=True) #setting multi-class as True gives multilabel 
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[0],reverse = False)) #sorting the dictionary of labels in alphabetical order based on their score
    t_score  = []
    #categories_score = []
    for i, (k,v) in enumerate(sorted_dict.items()):
      #categories_score.append(k)
      t_score.append(v)
    tweet_scores.append(t_score) #return all scores with categories sorted alphabetical order
   # categories_scores.append(categories_score)
    categories  = []
    sorted_dict_ = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True))
    for i, (k,v) in enumerate(sorted_dict_.items()):
      if(i > 2): #return categories with only the best 3 predictions
        break
      else:
        categories.append(k)
    predictedCategories.append(categories)

  4%|▍         | 10/245 [00:01<00:37,  6.26it/s]/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:901: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
100%|██████████| 245/245 [00:40<00:00,  6.01it/s]


In [ ]:
#Put it all together
tweet_scores_df = pd.DataFrame(tweet_scores, columns=['Activism','Lekki Massacre','Media coverage','No response from public figures','Police Brutality','Questions about moving forward/past','Irrelevant'])
tweet_scores_df

,Activism,Lekki Massacre,Media coverage,No response from public figures,Police Brutality,Questions about moving forward/past,Irrelevant
0,0.754780,0.000148,0.012566,0.122678,0.000159,0.039525,0.396220
1,0.025495,0.001117,0.048469,0.018139,0.003023,0.047655,0.189236
2,0.014667,0.003151,0.006352,0.035611,0.001678,0.000117,0.949163
3,0.011175,0.000397,0.013504,0.004388,0.000089,0.000939,0.247348
4,0.289728,0.000376,0.034569,0.339679,0.000332,0.089706,0.334366
...,...,...,...,...,...,...,...
240,0.121981,0.000145,0.004028,0.012895,0.000226,0.000634,0.660356
241,0.959354,0.000257,0.034022,0.039049,0.000395,0.214194,0.383004
242,0.249530,0.000489,0.011538,0.249910,0.019596,0.928222,0.675362
243,0.145861,0.000235,0.000386,0.061604,0.000102,0.578648,0.370735


In [ ]:
#Select the  top-3 most confident predictions
tweet_label_df = pd.DataFrame(predictedCategories, columns=['Label 1','Label 2','Label 3'])
tweet_label_df

,Label 1,Label 2,Label 3
0,Activism,Questions about moving forward/past,Media coverage
1,Questions about moving forward/past,Lekki Massacre,Police brutality events
2,Questions about moving forward/past,Media coverage,Activism
3,Questions about moving forward/past,Lekki Massacre,Activism
4,Media coverage,Questions about moving forward/past,Activism
...,...,...,...
240,Questions about moving forward/past,Activism,Media coverage
241,Activism,Questions about moving forward/past,Police brutality events
242,Police brutality events,Questions about moving forward/past,Media coverage
243,Police brutality events,Questions about moving forward/past,Activism


In [ ]:
#Combine all labels
tweet_label_df['Predicted label'] = tweet_label_df[['Label 1','Label 2','Label 3']].apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)
y_ZeroShot = tweet_label_df['Predicted label']
y_ZeroShot_multilabel = vectorizer.transform(y_ZeroShot)
y_ZeroShot_multilabel

<245x7 sparse matrix of type '<class 'numpy.int64'>'
	with 735 stored elements in Compressed Sparse Row format>

In [ ]:
print("Accuracy :",metrics.accuracy_score(y_test_multilabel, y_ZeroShot_multilabel))
print("Hamming loss ",metrics.hamming_loss(y_test_multilabel,y_ZeroShot_multilabel))

precision = precision_score(y_test_multilabel, y_ZeroShot_multilabel, average='micro')
recall = recall_score(y_test_multilabel, y_ZeroShot_multilabel, average='micro')
f1 = f1_score(y_test_multilabel, y_ZeroShot_multilabel, average='micro')
 
print("\nMicro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test_multilabel, y_ZeroShot_multilabel, average='macro')
recall = recall_score(y_test_multilabel, y_ZeroShot_multilabel, average='macro')
f1 = f1_score(y_test_multilabel, y_ZeroShot_multilabel, average='macro')
 
print("\nMacro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

print("\nClassification Report")
print (metrics.classification_report(y_test_multilabel, y_ZeroShot_multilabel))
# print("Time taken to run this cell :", datetime.now() - start)

Accuracy : 0.02040816326530612
Hamming loss  0.3434402332361516

Micro-average quality numbers
Precision: 0.3184, Recall: 0.7267, F1-measure: 0.4428

Macro-average quality numbers
Precision: 0.3698, Recall: 0.6607, F1-measure: 0.3492

Classification Report
              precision    recall  f1-score   support

           0       0.68      0.92      0.78       154
           1       0.80      0.08      0.14        53
           2       0.30      0.76      0.43        21
           3       0.12      0.89      0.21        18
           4       0.31      0.25      0.28        16
           5       0.25      0.76      0.38        29
           6       0.13      0.97      0.22        31

   micro avg       0.32      0.73      0.44       322
   macro avg       0.37      0.66      0.35       322
weighted avg       0.53      0.73      0.51       322
 samples avg       0.32      0.70      0.42       322



In [ ]:
tweet_data_plus_labels  = {'tweet_id':test_data['tweet_id'].to_list(),'Original_Tweets':test_data['tweet'].to_list(),'Cleaned_Tweets':test_data['cleaned_tweets'].to_list(),'Human_label':test_data['HumanLabel'].to_list(), 'Zero_shot_label':tweet_label_df['Predicted label'].to_list()}
Zero_shot_df = pd.DataFrame(tweet_data_plus_labels,pd.set_option('display.max_colwidth', -1))
Zero_shot_df.head(2)

In [ ]:
top3_USE

,1st Max,2nd Max,3rd Max
0,Activism,No Response from public figures,Questions about moving forward/past
1,No Response from public figures,Lekki Massacre,Irrelevant
2,Lekki Massacre,Activism,No Response from public figures
3,No Response from public figures,Police brutality events,Media coverage
4,Activism,Lekki Massacre,Questions about moving forward/past
...,...,...,...
240,No Response from public figures,Irrelevant,Police brutality events
241,Activism,Lekki Massacre,Media coverage
242,No Response from public figures,Irrelevant,Police brutality events
243,Media coverage,Lekki Massacre,Questions about moving forward/past
